In [3]:
import wrds
import pandas as pd

In [4]:
db = wrds.Connection(wrds_username='ayansola')
# setup pg_pass needed for access to the wrds dataset (first time only)
# db.create_pgpass_file()

Loading library list...
Done


In [5]:
params = {
    "from_date": "2005-01-01",
    "to_date": "2023-12-31"
}

t_bill = db.raw_sql(
    """
    SELECT date, dgs2 as treasury_2y
    FROM frb.rates_daily
    WHERE date BETWEEN %(from_date)s AND %(to_date)s
    """,
    params=params,
)

In [6]:
t_bill['date'] = pd.to_datetime(t_bill['date'])

In [7]:
t_bill.sample(5)

,date,treasury_2y
1960,2010-05-15,<NA>
1146,2008-02-21,1.98
4718,2017-12-02,<NA>
3075,2013-06-03,0.3
1673,2009-08-01,<NA>


In [9]:
t_bill['year'] = pd.to_datetime(t_bill['date']).dt.year

In [10]:
# Write to Parquet with partitioning
t_bill.to_parquet(
    "tbills/",
    engine="pyarrow",        # or "fastparquet"
    partition_cols=["year"],
    index=False
)